In [3]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
import matplotlib.pyplot as plt 
from sklearn.utils import shuffle
import warnings
warnings.filterwarnings('ignore')
import sklearn.metrics

In [4]:
from tqdm import tqdm 

### DATASET = Original + Augmented

### MIC

### ALL

In [5]:
orig_loc = "./Data_CSV/Experiment_W_WO_CMN/"
aug_loc = "./Data_CSV/Augmneted_MFCC_data_csv/"

In [6]:
#without CMN #original_data
mic_all_orig_df = pd.read_csv("Data_CSV/Experiment_W_WO_CMN/Without_CMN_mic(16k)_all.csv")
mic_bld_orig_df = pd.read_csv("Data_CSV/Experiment_W_WO_CMN/withoutCMN(all)micB_L_D_DPB.csv")
mic_blt_orig_df = pd.read_csv("Data_CSV/Experiment_W_WO_CMN/withoutCMN(all)micB_L_T_DPB.csv")
mic_brd_orig_df = pd.read_csv("Data_CSV/Experiment_W_WO_CMN/withoutCMN(all)micB_R_D_DPB.csv")
mic_brt_orig_df = pd.read_csv("Data_CSV/Experiment_W_WO_CMN/withoutCMN(all)micB_R_T_DPB.csv")

In [7]:
print(f"Shape: {mic_all_orig_df.shape}")
# mic_all_orig_df.head(5)

Shape: (4843, 74)


#### Augmented Data

In [8]:
mic_bld_aug_df = pd.read_csv("./Data_CSV/Augmneted_MFCC_data_csv/micB_L_D_DPB.csv")
mic_blt_aug_df = pd.read_csv("./Data_CSV/Augmneted_MFCC_data_csv/micB_L_T_DPB.csv")
mic_brd_aug_df = pd.read_csv("./Data_CSV/Augmneted_MFCC_data_csv/micB_R_D_DPB.csv")
mic_brt_aug_df = pd.read_csv("./Data_CSV/Augmneted_MFCC_data_csv/micB_R_T_DPB.csv")
mic_all_aug_df = pd.concat([mic_bld_aug_df,mic_blt_aug_df,mic_brd_aug_df,mic_brt_aug_df])

In [9]:
mic_all_aug_df.shape

(14514, 74)

In [10]:
#fold
def fold(sound_dataframe, sets_loc, fold_type):
    big = []
    for j in os.listdir(sets_loc):      

        file = open(sets_loc+j, 'r') 
        Lines = file.readlines() 

        count = 0
        # Strips the newline character 
        # for line in Lines: 
        #     print("Line{}: {}".format(count, line.strip())) 
        try:
            set_dataframe = pd.DataFrame()
            for i, line in enumerate(Lines):
                if fold_type == 'original':
                    A = sound_dataframe[(sound_dataframe['73'] == (Lines[i].strip()+".json"))]
                else:
                    A = sound_dataframe[(sound_dataframe['73'] == ("Aug"+Lines[i].strip()+".json"))]
                set_dataframe = pd.DataFrame.append(set_dataframe,A)
        except:
            pass
            
        
        big.append(set_dataframe)
    fold_1 = shuffle(pd.concat(big[:6]))
    test_1 = shuffle(big[6])
    fold_2 = shuffle(pd.concat(big[1:7]))
    test_2 = shuffle(big[0])
    fold_3 = shuffle(pd.concat([big[0],big[2],big[3],big[4], big[5], big[6]]))
    test_3 = shuffle(big[1])
    fold_4 = shuffle(pd.concat([big[0],big[1],big[3],big[4], big[5], big[6]]))
    test_4 = shuffle(big[2])
    fold_5 = shuffle(pd.concat([big[0],big[2],big[1],big[4], big[5], big[6]]))
    test_5 = shuffle(big[3])
    fold_6 = shuffle(pd.concat([big[0],big[2],big[1],big[3], big[5], big[6]]))
    test_6 = shuffle(big[4])
    fold_7 = shuffle(pd.concat([big[0],big[2],big[1],big[4], big[3], big[6]]))
    test_7 = shuffle(big[5])
    
    return fold_1,test_1,fold_2,test_2,fold_3,test_3,fold_4,test_4,fold_5,test_5, fold_6, test_6, fold_7,test_7

In [11]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5,f6,t6,f7,t7 = fold(mic_all_orig_df, './Sets/', 'original')
f1_aug,t1_aug,f2_aug,t2_aug,f3_aug,t3_aug,f4_aug,t4_aug,f5_aug,t5_aug,f6_aug,t6_aug,f7_aug,t7_aug = fold(mic_all_aug_df, './Sets/', 'augmented')


In [12]:
fold_list = [shuffle(pd.concat([f1,f1_aug])),t1,shuffle(pd.concat([f2,f2_aug])),t2,shuffle(pd.concat([f3,f3_aug])),t3,shuffle(pd.concat([f4,f4_aug])),t4,shuffle(pd.concat([f5,f5_aug])),t5,shuffle(pd.concat([f6,f6_aug])),t6,shuffle(pd.concat([f7,f7_aug])),t7 ]

In [13]:
def SVM(fold_list):
    list_train = []
    list_test = []
    for i in tqdm(range(0,len(fold_list),2)):
        fold = fold_list[i]
        test = fold_list[i+1]
        X_train = fold.iloc[:,:72]
        y_train = fold.iloc[:,72]
        X_test = test.iloc[:,:72]
        y_test = test.iloc[:,72]

        scaler = StandardScaler()

        X_train = scaler.fit_transform(X_train)

        X_test = scaler.transform(X_test)

        X_train = pd.DataFrame(X_train)
        X_test = pd.DataFrame(X_test)


        ### SVM

        svm = SVC()

        svm.fit(X_train,y_train)

        list_test.append(accuracy_score(y_test,svm.predict(X_test)))

        list_train.append(accuracy_score(y_train,svm.predict(X_train)))
        
    return list_test, list_train

In [149]:
test, train = SVM(fold_list)

100%|██████████| 7/7 [01:47<00:00, 15.59s/it]


In [150]:
test.append(np.mean(test))
test.append(np.std(test[:7]))

train.append(np.mean(train))
train.append(np.std(train[:7]))

In [151]:
pd.DataFrame(np.concatenate((np.array(train).reshape(9,1), np.array(test).reshape(9,1)), axis = 1), index=['Fold1','Fold2','Fold3','Fold4','Fold5','Fold6','Fold7', 'Mean', 'STD'], columns=['Train', 'Test'])

,Train,Test
Fold1,0.987673,0.390995
Fold2,0.990158,0.565093
Fold3,0.986931,0.606476
Fold4,0.986963,0.491409
Fold5,0.987394,0.573569
Fold6,0.990641,0.359375
Fold7,0.988380,0.672646
Mean,0.988306,0.522795
STD,0.001405,0.106158


### BRT_MIC

In [14]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5,f6,t6,f7,t7 = fold(mic_brt_orig_df, './Sets/', 'original')
f1_aug,t1_aug,f2_aug,t2_aug,f3_aug,t3_aug,f4_aug,t4_aug,f5_aug,t5_aug,f6_aug,t6_aug,f7_aug,t7_aug = fold(mic_brt_aug_df, './Sets/', 'augmented')
fold_list = [shuffle(pd.concat([f1,f1_aug])),t1,shuffle(pd.concat([f2,f2_aug])),t2,shuffle(pd.concat([f3,f3_aug])),t3,shuffle(pd.concat([f4,f4_aug])),t4,shuffle(pd.concat([f5,f5_aug])),t5,shuffle(pd.concat([f6,f6_aug])),t6,shuffle(pd.concat([f7,f7_aug])),t7 ]

In [16]:
test, train = SVM(fold_list)
test.append(np.mean(test))
test.append(np.std(test[:7]))

train.append(np.mean(train))
train.append(np.std(train[:7]))
pd.DataFrame(np.concatenate((np.array(train).reshape(9,1), np.array(test).reshape(9,1)), axis = 1), index=['Fold1','Fold2','Fold3','Fold4','Fold5','Fold6','Fold7', 'Mean', 'STD'], columns=['Train', 'Test'])

100%|██████████| 7/7 [00:05<00:00,  1.18it/s]


,Train,Test
Fold1,0.994588,0.447826
Fold2,0.994054,0.518325
Fold3,0.993648,0.583333
Fold4,0.992730,0.552239
Fold5,0.990935,0.611842
Fold6,0.994656,0.604839
Fold7,0.992598,0.666667
Mean,0.993315,0.569296
STD,0.001230,0.065896


### BRD_MIC

In [17]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5,f6,t6,f7,t7 = fold(mic_brd_orig_df, './Sets/', 'original')
f1_aug,t1_aug,f2_aug,t2_aug,f3_aug,t3_aug,f4_aug,t4_aug,f5_aug,t5_aug,f6_aug,t6_aug,f7_aug,t7_aug = fold(mic_brd_aug_df, './Sets/', 'augmented')
fold_list = [shuffle(pd.concat([f1,f1_aug])),t1,shuffle(pd.concat([f2,f2_aug])),t2,shuffle(pd.concat([f3,f3_aug])),t3,shuffle(pd.concat([f4,f4_aug])),t4,shuffle(pd.concat([f5,f5_aug])),t5,shuffle(pd.concat([f6,f6_aug])),t6,shuffle(pd.concat([f7,f7_aug])),t7 ]

In [18]:
test, train = SVM(fold_list)
test.append(np.mean(test))
test.append(np.std(test[:7]))

train.append(np.mean(train))
train.append(np.std(train[:7]))
pd.DataFrame(np.concatenate((np.array(train).reshape(9,1), np.array(test).reshape(9,1)), axis = 1), index=['Fold1','Fold2','Fold3','Fold4','Fold5','Fold6','Fold7', 'Mean', 'STD'], columns=['Train', 'Test'])

100%|██████████| 7/7 [00:06<00:00,  1.08it/s]


,Train,Test
Fold1,0.994500,0.463768
Fold2,0.991784,0.676056
Fold3,0.991063,0.715116
Fold4,0.992850,0.468354
Fold5,0.992530,0.453202
Fold6,0.993662,0.387324
Fold7,0.992920,0.628415
Mean,0.992758,0.541748
STD,0.001053,0.118780


### BLT_Mic

In [19]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5,f6,t6,f7,t7 = fold(mic_blt_orig_df, './Sets/', 'original')
f1_aug,t1_aug,f2_aug,t2_aug,f3_aug,t3_aug,f4_aug,t4_aug,f5_aug,t5_aug,f6_aug,t6_aug,f7_aug,t7_aug = fold(mic_blt_aug_df, './Sets/', 'augmented')
fold_list = [shuffle(pd.concat([f1,f1_aug])),t1,shuffle(pd.concat([f2,f2_aug])),t2,shuffle(pd.concat([f3,f3_aug])),t3,shuffle(pd.concat([f4,f4_aug])),t4,shuffle(pd.concat([f5,f5_aug])),t5,shuffle(pd.concat([f6,f6_aug])),t6,shuffle(pd.concat([f7,f7_aug])),t7 ]

In [20]:
test, train = SVM(fold_list)
test.append(np.mean(test))
test.append(np.std(test[:7]))

train.append(np.mean(train))
train.append(np.std(train[:7]))
pd.DataFrame(np.concatenate((np.array(train).reshape(9,1), np.array(test).reshape(9,1)), axis = 1), index=['Fold1','Fold2','Fold3','Fold4','Fold5','Fold6','Fold7', 'Mean', 'STD'], columns=['Train', 'Test'])

100%|██████████| 7/7 [00:06<00:00,  1.05it/s]


,Train,Test
Fold1,0.995441,0.316279
Fold2,0.994787,0.554455
Fold3,0.991627,0.627358
Fold4,0.992806,0.606897
Fold5,0.992544,0.657609
Fold6,0.994889,0.287879
Fold7,0.991514,0.736527
Mean,0.993373,0.541001
STD,0.001516,0.159633


### BLD_MIC

In [21]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5,f6,t6,f7,t7 = fold(mic_bld_orig_df, './Sets/', 'original')
f1_aug,t1_aug,f2_aug,t2_aug,f3_aug,t3_aug,f4_aug,t4_aug,f5_aug,t5_aug,f6_aug,t6_aug,f7_aug,t7_aug = fold(mic_bld_aug_df, './Sets/', 'augmented')
fold_list = [shuffle(pd.concat([f1,f1_aug])),t1,shuffle(pd.concat([f2,f2_aug])),t2,shuffle(pd.concat([f3,f3_aug])),t3,shuffle(pd.concat([f4,f4_aug])),t4,shuffle(pd.concat([f5,f5_aug])),t5,shuffle(pd.concat([f6,f6_aug])),t6,shuffle(pd.concat([f7,f7_aug])),t7 ]

In [22]:
test, train = SVM(fold_list)
test.append(np.mean(test))
test.append(np.std(test[:7]))

train.append(np.mean(train))
train.append(np.std(train[:7]))
pd.DataFrame(np.concatenate((np.array(train).reshape(9,1), np.array(test).reshape(9,1)), axis = 1), index=['Fold1','Fold2','Fold3','Fold4','Fold5','Fold6','Fold7', 'Mean', 'STD'], columns=['Train', 'Test'])

100%|██████████| 7/7 [00:06<00:00,  1.16it/s]


,Train,Test
Fold1,0.991599,0.463542
Fold2,0.991832,0.676829
Fold3,0.992019,0.522167
Fold4,0.989553,0.413793
Fold5,0.989785,0.492308
Fold6,0.993632,0.228070
Fold7,0.991609,0.478261
Mean,0.991433,0.467853
STD,0.001290,0.124076


### EKO

### ALL

In [24]:
#without CMN #original_data
eko_all_orig_df  = pd.read_csv("Data_CSV/Experiment_W_WO_CMN/Without_CMN_eko_all.csv")
eko_bld_orig_df  = pd.read_csv("Data_CSV/Experiment_W_WO_CMN/withoutCMN(all)ekoB_L_D_DPB.csv")
eko_blt_orig_df  = pd.read_csv("Data_CSV/Experiment_W_WO_CMN/withoutCMN(all)ekoB_L_T_DPB.csv")
eko_brd_orig_df  = pd.read_csv("Data_CSV/Experiment_W_WO_CMN/withoutCMN(all)ekoB_R_D_DPB.csv")
eko_brt_orig_df  = pd.read_csv("Data_CSV/Experiment_W_WO_CMN/withoutCMN(all)ekoB_R_T_DPB.csv")
eko_bld_aug_df   = pd.read_csv("./Data_CSV/Augmneted_MFCC_data_csv/ekoB_L_D_DPB.csv")
eko_blt_aug_df   = pd.read_csv("./Data_CSV/Augmneted_MFCC_data_csv/ekoB_L_T_DPB.csv")
eko_brd_aug_df   = pd.read_csv("./Data_CSV/Augmneted_MFCC_data_csv/ekoB_R_D_DPB.csv")
eko_brt_aug_df   = pd.read_csv("./Data_CSV/Augmneted_MFCC_data_csv/ekoB_R_T_DPB.csv")
eko_all_aug_df   = pd.concat([eko_bld_aug_df,eko_blt_aug_df,eko_brd_aug_df,eko_brt_aug_df]) 

In [154]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5,f6,t6,f7,t7 = fold(eko_all_orig_df, './Sets/', 'original')
f1_aug,t1_aug,f2_aug,t2_aug,f3_aug,t3_aug,f4_aug,t4_aug,f5_aug,t5_aug,f6_aug,t6_aug,f7_aug,t7_aug = fold(eko_all_aug_df, './Sets/', 'augmented')


In [155]:
fold_list = [shuffle(pd.concat([f1,f1_aug])),t1,shuffle(pd.concat([f2,f2_aug])),t2,shuffle(pd.concat([f3,f3_aug])),t3,shuffle(pd.concat([f4,f4_aug])),t4,shuffle(pd.concat([f5,f5_aug])),t5,shuffle(pd.concat([f6,f6_aug])),t6,shuffle(pd.concat([f7,f7_aug])),t7 ]

In [156]:
test_eko, train_eko = SVM(fold_list)

100%|██████████| 7/7 [00:51<00:00,  7.44s/it]


In [157]:
test_eko.append(np.mean(test_eko))
test_eko.append(np.std(test_eko[:7]))

train_eko.append(np.mean(train_eko))
train_eko.append(np.std(train_eko[:7]))

In [159]:
pd.DataFrame(np.concatenate((np.array(train_eko).reshape(9,1), np.array(test_eko).reshape(9,1)), axis = 1), index=['Fold1','Fold2','Fold3','Fold4','Fold5','Fold6','Fold7', 'Mean', 'STD'], columns=['Train', 'Test'])

,Train,Test
Fold1,0.902790,0.560427
Fold2,0.902934,0.596567
Fold3,0.901176,0.485679
Fold4,0.903252,0.678694
Fold5,0.894314,0.678474
Fold6,0.916022,0.523438
Fold7,0.905726,0.626308
Mean,0.903745,0.592798
STD,0.006000,0.068857


### BRT_eko

In [160]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5,f6,t6,f7,t7 = fold(eko_brt_orig_df, './Sets/', 'original')
f1_aug,t1_aug,f2_aug,t2_aug,f3_aug,t3_aug,f4_aug,t4_aug,f5_aug,t5_aug,f6_aug,t6_aug,f7_aug,t7_aug = fold(eko_brt_aug_df, './Sets/', 'augmented')


In [164]:
fold_list = [shuffle(pd.concat([f1,f1_aug])),t1,shuffle(pd.concat([f2,f2_aug])),t2,shuffle(pd.concat([f3,f3_aug])),t3,shuffle(pd.concat([f4,f4_aug])),t4,shuffle(pd.concat([f5,f5_aug])),t5,shuffle(pd.concat([f6,f6_aug])),t6,shuffle(pd.concat([f7,f7_aug])),t7 ]

In [165]:
test, train = SVM(fold_list)
test.append(np.mean(test))
test.append(np.std(test[:7]))

train.append(np.mean(train))
train.append(np.std(train[:7]))
pd.DataFrame(np.concatenate((np.array(train).reshape(9,1), np.array(test).reshape(9,1)), axis = 1), index=['Fold1','Fold2','Fold3','Fold4','Fold5','Fold6','Fold7', 'Mean', 'STD'], columns=['Train', 'Test'])

100%|██████████| 7/7 [00:02<00:00,  2.31it/s]


,Train,Test
Fold1,0.947938,0.604348
Fold2,0.933598,0.738220
Fold3,0.933943,0.648148
Fold4,0.938086,0.589552
Fold5,0.927481,0.598684
Fold6,0.943773,0.653226
Fold7,0.936008,0.673203
Mean,0.937261,0.643626
STD,0.006310,0.048435


### BRD_eko

In [25]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5,f6,t6,f7,t7 = fold(eko_brd_orig_df, './Sets/', 'original')
f1_aug,t1_aug,f2_aug,t2_aug,f3_aug,t3_aug,f4_aug,t4_aug,f5_aug,t5_aug,f6_aug,t6_aug,f7_aug,t7_aug = fold(eko_brd_aug_df, './Sets/', 'augmented')
fold_list = [shuffle(pd.concat([f1,f1_aug])),t1,shuffle(pd.concat([f2,f2_aug])),t2,shuffle(pd.concat([f3,f3_aug])),t3,shuffle(pd.concat([f4,f4_aug])),t4,shuffle(pd.concat([f5,f5_aug])),t5,shuffle(pd.concat([f6,f6_aug])),t6,shuffle(pd.concat([f7,f7_aug])),t7 ]
test, train = SVM(fold_list)
test.append(np.mean(test))
test.append(np.std(test[:7]))

train.append(np.mean(train))
train.append(np.std(train[:7]))
pd.DataFrame(np.concatenate((np.array(train).reshape(9,1), np.array(test).reshape(9,1)), axis = 1), index=['Fold1','Fold2','Fold3','Fold4','Fold5','Fold6','Fold7', 'Mean', 'STD'], columns=['Train', 'Test'])

100%|██████████| 7/7 [00:03<00:00,  2.06it/s]


,Train,Test
Fold1,0.943500,0.714976
Fold2,0.944601,0.450704
Fold3,0.940097,0.424419
Fold4,0.932793,0.746835
Fold5,0.930777,0.482759
Fold6,0.945540,0.521127
Fold7,0.946289,0.459016
Mean,0.940514,0.542834
STD,0.005841,0.122396


### BLT_eko

In [26]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5,f6,t6,f7,t7 = fold(eko_blt_orig_df, './Sets/', 'original')
f1_aug,t1_aug,f2_aug,t2_aug,f3_aug,t3_aug,f4_aug,t4_aug,f5_aug,t5_aug,f6_aug,t6_aug,f7_aug,t7_aug = fold(eko_blt_aug_df, './Sets/', 'augmented')
fold_list = [shuffle(pd.concat([f1,f1_aug])),t1,shuffle(pd.concat([f2,f2_aug])),t2,shuffle(pd.concat([f3,f3_aug])),t3,shuffle(pd.concat([f4,f4_aug])),t4,shuffle(pd.concat([f5,f5_aug])),t5,shuffle(pd.concat([f6,f6_aug])),t6,shuffle(pd.concat([f7,f7_aug])),t7 ]
test, train = SVM(fold_list)
test.append(np.mean(test))
test.append(np.std(test[:7]))

train.append(np.mean(train))
train.append(np.std(train[:7]))
pd.DataFrame(np.concatenate((np.array(train).reshape(9,1), np.array(test).reshape(9,1)), axis = 1), index=['Fold1','Fold2','Fold3','Fold4','Fold5','Fold6','Fold7', 'Mean', 'STD'], columns=['Train', 'Test'])

100%|██████████| 7/7 [00:03<00:00,  1.91it/s]


,Train,Test
Fold1,0.936660,0.553488
Fold2,0.953555,0.465347
Fold3,0.947368,0.292453
Fold4,0.950090,0.648276
Fold5,0.940820,0.565217
Fold6,0.959111,0.393939
Fold7,0.954587,0.700599
Mean,0.948885,0.517046
STD,0.007340,0.132466


### BLD_eko 

In [27]:
f1,t1,f2,t2,f3,t3,f4,t4,f5,t5,f6,t6,f7,t7 = fold(eko_bld_orig_df, './Sets/', 'original')
f1_aug,t1_aug,f2_aug,t2_aug,f3_aug,t3_aug,f4_aug,t4_aug,f5_aug,t5_aug,f6_aug,t6_aug,f7_aug,t7_aug = fold(eko_bld_aug_df, './Sets/', 'augmented')
fold_list = [shuffle(pd.concat([f1,f1_aug])),t1,shuffle(pd.concat([f2,f2_aug])),t2,shuffle(pd.concat([f3,f3_aug])),t3,shuffle(pd.concat([f4,f4_aug])),t4,shuffle(pd.concat([f5,f5_aug])),t5,shuffle(pd.concat([f6,f6_aug])),t6,shuffle(pd.concat([f7,f7_aug])),t7 ]
test, train = SVM(fold_list)
test.append(np.mean(test))
test.append(np.std(test[:7]))


train.append(np.mean(train))
train.append(np.std(train[:7]))
pd.DataFrame(np.concatenate((np.array(train).reshape(9,1), np.array(test).reshape(9,1)), axis = 1), index=['Fold1','Fold2','Fold3','Fold4','Fold5','Fold6','Fold7', 'Mean', 'STD'], columns=['Train', 'Test'])

100%|██████████| 7/7 [00:03<00:00,  2.19it/s]


,Train,Test
Fold1,0.947047,0.427083
Fold2,0.950990,0.597561
Fold3,0.949537,0.334975
Fold4,0.941691,0.620690
Fold5,0.946374,0.651282
Fold6,0.960377,0.421053
Fold7,0.956071,0.503106
Mean,0.950298,0.507964
STD,0.005807,0.110401


### EKO(not_all)